In [1]:
from ipynb.fs.full.feature_engineering_tree import X_Transformer
from ipynb.fs.full.feature_engineering_tree import TargetNormalizedRegressor
import pandas as pd
from ipynb.fs.full.feature_engineering_tree import DataLoader
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
import numpy.ma as ma
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
from mlxtend.evaluate import paired_ttest_5x2cv
import seaborn as sns

In [2]:
def rmsle(y_true, y_pred, sample_weight=None, multioutput='uniform_average'):
    return np.sqrt(mean_squared_log_error(y_true, y_pred, sample_weight, multioutput))
    
#I just want to make rmse positive
flipped_score = make_scorer(rmsle, greater_is_better=True)

In [3]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5),scoring=None):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [4]:
data_loader = DataLoader("train.csv","test.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [5]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()

In [6]:
x_transformer = X_Transformer()

In [7]:
X_train = x_transformer.fit_transform(X_train,np.log1p(Y_train))
kf = KFold(n_splits=10,shuffle = True,random_state = 42)

In [8]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,12.039367,12.044987,12.179778,11.959059,12.303263,12.045973,11.810708,12.162162,12.071050,12.173153,...,4006.0,34.936386,12.146147,3.5,18.729746,0.0,1.0,1.0,1.0,0.0
1,12.039469,12.045093,12.180003,12.371039,11.955508,12.046084,12.189454,12.162322,12.071156,12.173483,...,3952.0,34.737418,11.973277,2.5,51.609318,0.0,0.0,1.0,1.0,1.0
2,12.039316,12.044933,12.179665,12.092858,12.303096,12.045917,12.188785,12.162081,12.070997,12.172987,...,4003.0,35.583983,12.005590,3.5,14.650192,0.0,1.0,1.0,1.0,1.0
3,12.039660,12.226079,11.815360,11.959479,11.956699,12.046292,12.259647,11.770364,12.071356,11.822254,...,3885.0,34.492754,11.640385,2.0,52.582659,0.0,1.0,1.0,1.0,1.0
4,12.039234,12.044847,12.179484,12.158223,12.302826,12.045827,12.188424,12.161952,12.070911,12.172720,...,4000.0,38.272092,12.531084,3.5,49.180733,0.0,1.0,1.0,1.0,1.0
5,12.039644,12.045276,12.180389,11.959457,12.304172,12.046275,11.811257,12.162597,12.071339,11.822219,...,3988.0,32.899728,11.820410,2.5,61.090014,0.0,1.0,1.0,1.0,0.0
6,12.039083,12.044690,12.656800,12.157285,12.302331,12.045663,12.257559,12.161715,12.070753,12.172231,...,4009.0,38.417063,12.697625,3.0,53.118619,0.0,0.0,1.0,1.0,1.0
7,12.039398,12.045018,12.179845,12.093841,11.955063,11.905233,12.189142,12.162210,12.071082,12.173252,...,3946.0,37.689115,12.633845,3.5,85.365813,0.0,1.0,1.0,1.0,1.0
8,12.039715,12.045349,11.815477,11.959558,11.969254,12.046351,12.190529,11.770559,11.672733,11.822379,...,3881.0,35.674617,11.448942,2.0,51.282346,0.0,1.0,1.0,1.0,1.0
9,12.039785,12.045423,11.815626,11.959659,12.304635,12.046428,12.190838,12.162819,12.261736,12.174508,...,3889.0,32.415644,11.634583,2.0,2.738417,0.0,0.0,1.0,1.0,1.0


In [ ]:
#cross validation and predicted vs measured cross validation
gbr_regressor_cross_val = TargetNormalizedRegressor(GradientBoostingRegressor(n_estimators=2000, learning_rate=0.01,
                                   max_depth=4, max_features='sqrt',
                                   loss='huber', random_state =42,subsample = 0.7))
scores = cross_val_score(gbr_regressor_cross_val, X_train, Y_train, cv=kf,n_jobs=-1,scoring = flipped_score)
print("Rmse log average: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))
print("Rmse log median: %0.5f" % (np.median(scores)))
predicted = cross_val_predict(gbr_regressor_cross_val, X_train, Y_train, cv=kf,n_jobs=-1)
print(scores)
fig, ax = plt.subplots()
ax.scatter(Y_train, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

Rmse log average: 0.03280 (+/- 0.02753)
Rmse log median: 0.03006


In [ ]:
#learning curve of best model
plot_learning_curve(gbr_regressor_cross_val, "Title", X_train, Y_train, None, cv=kf, n_jobs=-1,scoring = flipped_score)